In [127]:
# -*- coding: utf-8 -*-
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
import jieba
import jieba.analyse
import numpy as np

jieba.set_dictionary('dict.txt.big')##加入繁體中文

totalMovie = 6000
Seg = [] #切詞後的結果
labels = []
labelType = []

def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords 

with open('movie.csv', encoding="utf-8") as file:
    stopwords = stopwordslist('StopWords.txt')  # 這裏加載停用詞的路徑 
    for i,line in enumerate(file.readlines()):
        
        if i == totalMovie:
            break
        storyText = line.split(" | ")
        story  = storyText[3]
        temp_text = re.findall('[\u4e00-\u9fa5]', story)
        story = ''.join(temp_text)
        
        jieba.add_word('為什麼')
        jieba.add_word('有沒有')
         
        tags = jieba.lcut(story,cut_all=False)
        ThisLine = ''
        for tag in tags:
            if tag not in stopwords:  
                if tag != '\t' and tag != '\n' and tag != '':  
                    ThisLine += tag  
                    ThisLine += " "   #再次組合成【帶空格】的串
                    
        Seg.append(ThisLine)
        
        label = storyText[1].split(" ")
        labels.append(label[0])

labelType = list(set(labels))
print(labelType)
print("Finish")

Building prefix dict from /Users/ruru/NLP/HW3/dict.txt.big ...
Loading model from cache /var/folders/7t/bb_qyqxs0s1d1slg3618dg8r0000gn/T/jieba.ubbcc4e0b36da5e3874b4ccf5fcbd7ff2.cache
Loading model cost 0.931 seconds.
Prefix dict has been built succesfully.


['科幻', '奇幻', '動作', '動畫', '音樂/歌舞', '溫馨/家庭', '戰爭', '勵志', '劇情', '冒險', '犯罪', '恐怖', '喜劇', '影展', '紀錄片', 'NA', '武俠', '戲劇', '歷史/傳記', '愛情', '懸疑/驚悚']
Finish


In [128]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(Seg))
#獲取詞袋模型中的所有詞語  
word=vectorizer.get_feature_names()
print("Finish")

Finish


In [135]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

train_data , test_data , train_label , test_label = train_test_split(tfidf,labels,train_size=5500, test_size=500)
knn = KNeighborsClassifier()
knn.fit(train_data,train_label)
Result = knn.predict(test_data)
print("Finish")

Finish


In [136]:
Score = 0
index = 0
with open('movie.csv', encoding="utf-8") as file:
    for i,line in enumerate(file.readlines()):
        if i == totalMovie:
            break
            
        AllText = line.split(" | ")
        nowLabel = AllText[1].split(" ")
        nowLabel = nowLabel[0]
        
        if i > totalMovie - len(Result):
            if nowLabel == Result[index]:
                Score += 1
            index += 1
            #print(i, nowLabel, Result[index])

print(Score, '/', len(Result))

70 / 500
